In [1]:
import numpy as np
import torch
import gc
from tqdm import tqdm, trange

from time import time

from qaoa.quantum_model import QuantumModel

In [3]:
class QAOA_torch:
    def __init__(self, hamiltonian, device="cpu"):
        self.hamiltonian = hamiltonian

        self.n = hamiltonian.num_qubits
        self.N = 2 ** self.n

        self.driver = torch.from_numpy(self.hamiltonian.driver.to_matrix()).to(device)
        self.mixer = torch.from_numpy(self.hamiltonian.mixer.to_matrix()).to(device)
        self.target = torch.from_numpy(self.hamiltonian.target.to_matrix()).to(device)
        
    def energy(self, gammas, betas):
        
        assert gammas.shape == betas.shape
        param_shape = gammas.shape

        if len(param_shape) == 1:
            batches = 1
            p = param_shape[0]
        elif len(gammas.shape) == 2:
            batches, p = param_shape
        else:
            raise RuntimeError("gammas and betas must be 1 or 2 dimensional")
        
        # (batches, N)
        psi = torch.ones(batches, self.N, dtype=torch.complex128) / np.sqrt(self.N)

        # (batches, p, 2)
        parameters = torch.stack((gammas, betas)).permute(1, 2, 0)

        # (2, N, N)
        hamiltonians = torch.stack((self.driver, self.mixer))

        # (batches, p, 2, N, N)
        hamiltonians = hamiltonians.unsqueeze(0).unsqueeze(0).repeat(batches, p, 1, 1, 1)

        hamiltonians = torch.einsum('bpc,bpcij->bpcij', parameters, hamiltonians)
        # (batches * p * 2, N, N)
        hamiltonians = hamiltonians.reshape(batches * p * 2, self.N, self.N)

        hamiltonians = torch.matrix_exp(hamiltonians * -1j)

        # (batches, p, 2, N, N)
        hamiltonians = hamiltonians.view(batches, p, 2, self.N, self.N)
        # (p, 2, batches, N, N)
        hamiltonians = hamiltonians.permute(1, 2, 0, 3, 4)

        for i in range(p):
            batch_driver, batch_mixer = hamiltonians[i]
            
            psi = torch.einsum('bij,bj->bi', batch_driver, psi)
            psi = torch.einsum('bij,bj->bi', batch_mixer, psi)
        
        del hamiltonians
        gc.collect()

        result = torch.einsum('ij,bj->bi', self.target, psi)
        result = torch.einsum('bi,bi->b', psi.conj(), result)

        return result.real

    def energy_landscape(self, res, p, chunk_size = None):
        s = torch.linspace(0, 2 * np.pi, res + 1)[:-1]
        coords = torch.meshgrid(*[s] * (2 * p))
        coords = torch.stack([coord.flatten() for coord in coords]).T
        gammas, betas = torch.hsplit(coords, 2)

        if chunk_size is None:
            gb_used = 20
            
            psi_size = self.N
            ham_size = p * 2 * self.N * self.N
            coord_size = (2 * p) * (self.N ** (2 * p))

            chunk_size = (gb_used / 4 * 1e9 - (16 * coord_size)) / (32 * (psi_size + ham_size))
            chunk_size = max(chunk_size, 10)

        num_chunks = int((res ** (2 * p)) / chunk_size)
        num_chunks = max(num_chunks, 1)
        
        gamma_chunks = torch.chunk(gammas, num_chunks)
        beta_chunks = torch.chunk(betas, num_chunks)

        # clear massive memory used
        del s
        del coords
        del gammas
        del betas
        gc.collect()

        energies = []
        for i in trange(num_chunks):
            energies.append(self.energy(gamma_chunks[i], beta_chunks[i]))

        energies = torch.cat(energies)
        energies = energies.reshape([res] * (2 * p))
        return energies

In [52]:
"""
Schwinger Model
"""

select = {'N': 4, #qubits
          'g' : 1,  #coupling
          'm' : 1,  #bare mass
          'a' : 1, #lattice spacing
          'theta' : 0, #topological term
          'mixer_type' : 'Y', # type of mixer {'X', 'Y', 'XY'}
         }

model = QuantumModel('schwinger', 'Standard')


ham = model.make(select)
schwinger_target = ham.target

torch_qaoa = QAOA_torch(ham)

ham

driver:
4.0 * IIII
+ 2.0 * IIZZ
+ 1.0 * IZIZ
+ 1.0 * IZZI
+ 4.0 * IIIZ
- 1.0 * IIZI
+ 3.0 * IZII
- 2.0 * ZIII
mixer:
1.0 * IIIY
+ 1.0 * IIYI
+ 1.0 * IYII
+ 1.0 * YIII
target:
1.0 * IIII
+ 0.5 * IIZZ
+ 0.25 * IZIZ
+ 0.25 * IZZI
+ 1.0 * IIIZ
- 0.25 * IIZI
+ 0.75 * IZII
- 0.5 * ZIII
+ 0.25 * IIXX
+ 0.25 * IIYY
+ 0.25 * IXXI
+ 0.25 * IYYI
+ 0.25 * XXII
+ 0.25 * YYII

In [51]:
from functools import lru_cache

@lru_cache
def repeated_tensor(base, exp):
    result = base

    if exp > 1:
        result = repeated_tensor(torch.kron(base, base), exp//2)
        if exp % 2 == 1:
            result = torch.kron(result, base)

    return result

def coupling_eig(coupling_tensor):
    eigvals = None

    pauli_diags = [torch.tensor([1, 1], dtype=torch.complex128), torch.tensor([1, -1], dtype=torch.complex128)]
    pauli_types = set()

    for ndx in coupling_tensor.keys():
        coeff = coupling_tensor[ndx]

        term = None

        for i in ndx:
            pauli_types.add(i)
            term = torch.kron(term, pauli_diags[min(i, 1)]) if term is not None else pauli_diags[min(i, 1)]
        term = term * coeff
        eigvals = eigvals + term if eigvals is not None else term

    if (1 in pauli_types and 2 in pauli_types) or (2 in pauli_types and 3 in pauli_types) or (3 in pauli_types and 1 in pauli_types):
        eigvals, eigvecs = torch.linalg.eigh(torch_qaoa.mixer)
        return eigvals.type(torch.complex128), eigvecs

    if 1 in pauli_types: # X
        eigvecs = torch.tensor([[1, 1], [1, -1]], dtype=torch.complex128) / np.sqrt(2)
    if 2 in pauli_types: # Y
        eigvecs = torch.tensor([[1, 1], [1j, -1j]], dtype=torch.complex128) / np.sqrt(2)
    else: # I and/or Z
        eigvecs = torch.tensor([[1, 0], [0, 1]], dtype=torch.complex128)

    eigvecs = repeated_tensor(eigvecs, coupling_tensor.n)

    return eigvals, eigvecs

eigvals, eigvecs = coupling_eig(ham.mixer_coupling)

In [37]:
def landscape_minimas(landscape, filter = True):

    dims = len(landscape.shape)
    res = landscape.shape[0]

    minimas = torch.full(landscape.shape, True)

    for dim in range(dims):
        less_right = landscape <= torch.roll(landscape, 1 , dim)
        less_left = landscape <= torch.roll(landscape, -1 , dim)

        axis_minimas = torch.logical_and(less_right, less_left)
        minimas = torch.logical_and(minimas, axis_minimas)

    if filter:
        global_minimas = torch.isclose(landscape, torch.min(landscape))
        minimas = torch.logical_and(minimas, global_minimas)

    min_energy = landscape[minimas]

    points = torch.stack(torch.where(minimas)).T * (2*np.pi) / res
    gammas, betas = torch.hsplit(points, 2)

    return gammas, betas, min_energy

In [38]:
gammas, betas, min_energies = landscape_minimas(energy_scape)

gammas = gammas.requires_grad_()
betas = betas.requires_grad_()

In [51]:
# optimizer = torch.optim.SGD([gammas, betas], lr=1e-3)
optimizer = torch.optim.Adam([gammas, betas], lr=1e-3)

grad_mag = np.infty

count = 0
while grad_mag > 1e-4:
    optimizer.zero_grad()
    energy = torch_qaoa.energy(gammas, betas)
    loss = torch.mean(energy)
    
    loss.backward()
    optimizer.step()

    grad_mag = max(torch.max(gammas.grad.abs()), torch.max(betas.grad.abs()))

    print(f"{count}: loss = {loss}, grad = {grad_mag}", end="\t\t\r")

    count += 1

In [52]:
torch.min(energy)

tensor(-2.0879, dtype=torch.float64, grad_fn=<MinBackward1>)

In [53]:
torch_qaoa.energy(gammas, betas)

tensor([-2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000,
        -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000,
        -2.0269, -2.0269, -2.0269, -2.0269, -2.0269, -2.0269, -2.0269, -2.0269,
        -2.0051, -2.0051, -2.0051, -2.0051, -2.0051, -2.0051, -2.0051, -2.0051,
        -2.0399, -2.0399, -2.0399, -2.0399, -2.0399, -2.0399, -2.0399, -2.0399,
        -2.0399, -2.0399, -2.0399, -2.0399, -2.0399, -2.0399, -2.0399, -2.0399,
        -2.0879, -2.0879, -2.0879, -2.0879, -2.0879, -2.0879, -2.0879, -2.0879,
        -2.0000, -2.0000, -2.0865, -2.0864, -2.0779, -2.0777, -2.0000, -2.0000,
        -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0710, -2.0710,
        -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0865, -2.0864,
        -2.0780, -2.0779, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000, -2.0000,
        -2.0000, -2.0000, -2.0710, -2.0710, -2.0000, -2.0000, -2.0000, -2.0000,
        -2.0710, -2.0710, -2.0710, -2.07

In [54]:
torch.count_nonzero(torch.isclose(energy, torch.min(energy)))

tensor(32)

In [7]:
batches = 100
p = 3

gammas = torch.rand(batches, p) * 2 * np.pi
betas = torch.rand(batches, p) * 2 * np.pi

gammas = gammas.requires_grad_()
betas = betas.requires_grad_()

In [50]:
tenant_water_3 = 153.99 + 192.78 + 179.85 + 192.78 + 166.92 + 208.03
tenant_water_2 = 201.20 + 160.59 + 187.67 + 186.79 + 158.75

tenant_electric_3 = 65.86 + 80.13 + 59.04 + 88.81 + 109.01 + 108.77 + 152.47 + 86.38 + 6.3 + 82.32 + 110.52 + 104.35
tenant_electric_2 =  107.28 + 80.5 + 35.55 + 109.93 + 185.87 + 205.26 + 232.78 + 138.36 + 116.95 + 74.44

total = (tenant_water_3 / 3) + (tenant_water_2 / 2) + (tenant_electric_3 / 3) + (tenant_electric_2 / 2)
print(total)

1807.0633333333333


In [34]:
X = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex128)
Y = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex128)
Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex128)
I = torch.tensor([[1, 0], [0, 1]], dtype=torch.complex128)

X_eigvecs = torch.tensor([[1, 1], [1, -1]], dtype=torch.complex128)
Y_eigvecs = torch.tensor([[1, 1], [1j, -1j]], dtype=torch.complex128)
Z_eigvecs = torch.tensor([[1, 0], [0, 1]], dtype=torch.complex128)

theta = torch.Tensor([1])
# for i in range(1_200_000):
#     torch.cos(theta) * torch.eye(2) - 1j * torch.sin(theta) * Y

In [5]:
eigvals, eigvecs = torch.linalg.eigh(X)  

eigvecs @ torch.diag(eigvals).type(torch.complex128) @ eigvecs.conj()

tensor([[2.2371e-17+0.j, 1.0000e+00+0.j],
        [1.0000e+00+0.j, 2.2371e-17+0.j]], dtype=torch.complex128)

In [6]:
eigvecs * np.sqrt(2)

tensor([[-1.+0.j,  1.+0.j],
        [ 1.+0.j,  1.+0.j]], dtype=torch.complex128)

In [38]:
from functools import lru_cache

@lru_cache
def repeated_tensor(base, exp):
    result = base

    if exp > 1:
        result = repeated_tensor(torch.kron(base, base), exp//2)
        if exp % 2 == 1:
            result = torch.kron(result, base)

    return result
    
repeated_tensor(X, 12).shape


torch.Size([4096, 4096])

In [23]:
torch.from_numpy(ham.driver.to_matrix()).diag()

tensor([12.+0.j, -2.+0.j,  8.+0.j,  2.+0.j,  2.+0.j, -8.+0.j,  2.+0.j,  0.+0.j, 16.+0.j,  2.+0.j,
        12.+0.j,  6.+0.j,  6.+0.j, -4.+0.j,  6.+0.j,  4.+0.j],
       dtype=torch.complex128)

In [24]:
ham.driver

PauliSumOp(SparsePauliOp(['IIII', 'IIZZ', 'IZIZ', 'IZZI', 'IIIZ', 'IIZI', 'IZII', 'ZIII'],
              coeffs=[ 4.+0.j,  2.+0.j,  1.+0.j,  1.+0.j,  4.+0.j, -1.+0.j,  3.+0.j, -2.+0.j]), coeff=1.0)

In [32]:
repeated_tensor(Z, 1).diag()

tensor([ 1.+0.j, -1.+0.j], dtype=torch.complex128)

In [30]:
# 000
# 001
# 010
# 011
# 100

In [59]:
torch.kron(Z, torch.kron(I, I)).diag()

tensor([ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j],
       dtype=torch.complex128)

In [47]:
size = 8
print(torch.arange(size) & 1)
print((torch.arange(size) >> 1) & 1)
print((torch.arange(size) >> 2) & 1)

tensor([0, 1, 0, 1, 0, 1, 0, 1])
tensor([0, 0, 1, 1, 0, 0, 1, 1])
tensor([0, 0, 0, 0, 1, 1, 1, 1])


tensor([[-4.9304e-32+0.0000e+00j,  0.0000e+00-1.0000e+00j,
          0.0000e+00-1.0000e+00j,  4.9304e-32+0.0000e+00j,
          0.0000e+00-1.0000e+00j,  4.9304e-32+0.0000e+00j,
          4.9304e-32+0.0000e+00j,  0.0000e+00+4.9304e-32j,
          0.0000e+00-1.0000e+00j,  4.9304e-32+0.0000e+00j,
          4.9304e-32+0.0000e+00j,  0.0000e+00+4.9304e-32j,
          4.9304e-32+0.0000e+00j,  0.0000e+00+4.9304e-32j,
          0.0000e+00+4.9304e-32j, -4.9304e-32+0.0000e+00j],
        [ 0.0000e+00+1.0000e+00j, -4.9304e-32+0.0000e+00j,
         -4.9304e-32+0.0000e+00j,  0.0000e+00-1.0000e+00j,
         -4.9304e-32+0.0000e+00j,  0.0000e+00-1.0000e+00j,
          0.0000e+00-4.9304e-32j,  4.9304e-32+0.0000e+00j,
         -4.9304e-32+0.0000e+00j,  0.0000e+00-1.0000e+00j,
          0.0000e+00-4.9304e-32j,  4.9304e-32+0.0000e+00j,
          0.0000e+00-4.9304e-32j,  4.9304e-32+0.0000e+00j,
          4.9304e-32+0.0000e+00j,  0.0000e+00+4.9304e-32j],
        [ 0.0000e+00+1.0000e+00j, -4.9304e-32+0.0000e+

In [19]:
ham.mixer.to_matrix()

array([[0.+0.j, 0.-1.j, 0.-1.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+1.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.-1.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.-1.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.-1.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.